In [1]:
import sys; sys.path.append('/rds/general/user/cb2714/home/wwa/'); from wwa import *

xn, xx, yn, yx = [-125,-110,30,40]

---
# MSWX

## Extract FWI variables

In [2]:
fpath = "/rds/general/user/cb2714/home/00_WWA_project_folder/live/data/mswx/"
outpath = "/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/cali-wildfires/mswx/"

In [3]:
for y in range(2025,1978,-1):
    print(y, end = " ")
    
    for varnm in ["P", "RelHum", "Tmax", "Wind"]:
        
        new_fnm = outpath+varnm+"_"+str(y)+".nc"
        if os.path.exists(new_fnm): continue
            
        print(varnm, end = " ")
        
        da = wrap_lon(xr.open_mfdataset(fpath+varnm+"/"+str(y)+"/*.nc")).sel(lon = slice(xn,xx), lat = slice(yn,yx))
        da.to_netcdf(new_fnm)
    print("")
clear_output(wait = False)
print("Done.")

Done.


## Compute FWI

In [3]:
from xclim.indicators.atmos import cffwis_indices

ds = xr.merge([xr.open_mfdataset(outpath+"*"+varnm+"*.nc") for varnm in ["P", "RelHum", "Tmax", "Wind"]]).load()
ds = ds.rename(precipitation = "pr", relative_humidity = "hurs", air_temperature = "tas", wind_speed = "sfcWind")

# mask to land surface only
ds = ds.where(regionmask.defined_regions.natural_earth_v5_0_0.land_50.mask(ds) == 0)

fwi = cffwis_indices(ds = ds, overwintering = False)
fwi = xr.merge(fwi)

fwi.to_netcdf(outpath+"fwi_mswx_1979-2025.nc")

/rds/general/user/cb2714/home/anaconda3/envs/wwa/lib/python3.10/site-packages/xclim/core/cfchecks.py:42: UserWarning: Variable does not have a `cell_methods` attribute.
  _check_cell_methods(
/rds/general/user/cb2714/home/anaconda3/envs/wwa/lib/python3.10/site-packages/xclim/core/cfchecks.py:46: UserWarning: Variable does not have a `standard_name` attribute.
  check_valid(vardata, "standard_name", data["standard_name"])
/rds/general/user/cb2714/home/anaconda3/envs/wwa/lib/python3.10/site-packages/xclim/indices/fire/_cffwis.py:840: RuntimeWarning: invalid value encountered in _drought_code
  out["DC"][..., it] = _drought_code(
/rds/general/user/cb2714/home/anaconda3/envs/wwa/lib/python3.10/site-packages/xclim/indices/fire/_cffwis.py:844: RuntimeWarning: invalid value encountered in _duff_moisture_code
  out["DMC"][..., it] = _duff_moisture_code(
/rds/general/user/cb2714/home/anaconda3/envs/wwa/lib/python3.10/site-packages/xclim/indices/fire/_cffwis.py:853: RuntimeWarning: invalid value